In [ ]:
import MDAnalysis as mda
import mdtraj as md
import numpy as np
import nglview as nv
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import math
import pickle
import os

def tanh_fit(z, dens):
    # Plot the average density profile along with best fit to tanh function with rho1>rho2 for the two coexisting phases
    def tanh_func(z, rho1, rho2, delta, z0):
        return 0.5*(rho1+rho2) - 0.5*(rho1-rho2) * np.tanh((z - z0) * 2.19722/delta)
    z1, z2 = np.split(z,2)
    rho1, rho2 = np.split(dens, 2)
    rho_av1 = (rho1[::-1]+ rho2[:])/2 # [::-1] means flip the array
    popt, pcov = curve_fit(tanh_func, z1, rho_av1, p0=[rho_av1.max(), rho_av1.min(), 2, z1.max()/2], maxfev=5000) # normal maxfev=1000
    rho_fit1= tanh_func(z1, *popt)
    return z1, rho_av1, rho_fit1, popt

def centered_dens(z,Lz):
        zx= np.cos(2*math.pi*z/Lz)
        zy= np.sin(2*math.pi*z/Lz)
        Cx= np.mean(zx)
        Cy= np.mean(zy)
        rad= np.arctan2(Cy,Cx) # [-pi,pi]
        if rad<0:
            rad+=2*math.pi
        zC= rad*Lz/(2*math.pi)
        r= np.sqrt(Cx**2+ Cy**2)
        return zC, r

In [ ]:
#### load trajectory as 'universe of mda' ####
# need psf and dcd files
folder= [] # can be changed
u = mda.Universe(folder+'/psfName.psf', folder+'/dcdName.dcd') # name can be changed

In [ ]:
#### parameters ####
c_nm= 0.1 # A -> nm
Lx, Ly, Lz= u.dimensions[:3]*c_nm # [A]
traj= u.trajectory
a= u.atoms

start= 0
period= 50 # less period is more accurate, but slower
nbins= 101
ratio_avg= 0.5 

dz= Lz/(nbins-1)
nT= math.floor(len(traj)) # total length of trajectory
nA= len(a) # number of atoms
    
'''
# --- average density profile
nFrame= 10000 # total recorded frame of simulation
timeTot= 500 # ! total time of simulation [ns]
end= math.floor(nFrame*len_ratio)
print(f"frame {start}:{period}:{end}")
'''

In [ ]:
#### accumulate process ####
dens_pep= []
for i in range(start, nT, period):
    traj[i]
    z = a.positions[:, 2]*c_nm
    zC, r= centered_dens(z,Lz)
    zR = (z- zC + Lz/2 )%Lz
    h1, binz = np.histogram(zR, bins = nbins-1, range=(0, Lz))
    rhoz= h1/(Lx*Ly*dz) # number density in [nm^{-3}]
    dens_pep.append(rhoz)
    zc= 0.5*(binz[:-1]+binz[1:])

nF= np.shape(dens_pep)[0] # number of picked frames
iBeg_avg= round(nF*(1-ratio_avg))
#
denM_pep= np.mean(dens_pep[iBeg_avg:,:],axis=0)

In [ ]:
#### fitting ####
zHf_pep, denHf_pep, denFit_pep, popt_pep= tanh_fit(zc,denM_pep)
denCod_pep= popt_pep[0]
denDil_pep= popt_pep[1]